<div style="width: 100%; background-color: #ef7d22; text-align: center">
<br><br>

<h1 style="color: white; font-weight: bold;">
    PostgreSQL Administration Tools
</h1>

<br><br> 
</div>

In concept, you could do all of your PostgreSQL administration via Python commands through an adapter. Nearly everything that can be configured can be configured via SQL commands.  Most likely, however, you would find this unecessarily cumbersome; using native administration tools is easier for most tasks.

Administration tools may be either command-line—i.e. the `psql` administration tool—or GUI tools.  In the latter category, *pgAdmin* is the default, and community-supported, tool.  pgAdmin can either provide a desktop GUI or create a localhost web server; in both cases though, your web browser is utilized for actually rendering the same interface.  pgAdmin is itself written in Python, but that footnote has little effect on how you use it.

A number of other Free Software and commercial tools are also available.  Many of the independent tools are designed to work with multiple RDBMS servers, and can potentially connect to multiple systems at the same time, even ones based on different server software.  These include OmniDB (https://omnidb.org/), DBeaver (https://dbeaver.io/), Pgweb (http://sosedoff.github.io/pgweb/).  Each of these tools have their own advantages and disadvantages, but in general they all look fairly similar and have largely the same capabilities.

Other GUI tools like Metabase, Redash, Blazer, Apache Superset, Apache Zepplin, and Franchise focus on so-called "business intelligence" (BI), which is about searching, visualizing, analyzing, and understanding the data inside PostgreSQL (or other) RDBMSs rather than administering them.  Since these tools, in concept, let you enter SQL commands, they could be used for administration, but that is not their focus.

---

We will talk a bit about pgAdmin in this lesson, but the greater part will be spent on psql because command-line tools enable better reproducibility and can be scripted more easily.

## pgAdmin
![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

It is not difficult to install pdAdmin, but details of doing so will vary slightly by platform.  Links to installers for various platforms can be found at:

> https://www.pgadmin.org/download/

The creators of the tool are kind enough to provide a trial version you can play with (and be guided through some lessons).  If you go to:

> https://www.pgadmin.org/try/

You can practice against test databases that they provide.  An initial dashboard gives some performance metrics.

<img src="https://user-images.githubusercontent.com/7065401/103534224-3efb7200-4e6d-11eb-86d5-bd88f41f4dec.png" width="50%" />

Rather than walk through numerous screenshots, I will perform a few simple tasks within the interface.  A login is required initially with the pgAdmin interface, with proper credentials.

* Select the `ine` database
* Unfold the tree to identify the Tables within it
* View 100 rows of `census_zipcode_geography` using popup menu
* Modify query using accesskey+I or toolbar (filter options), such that we only choose rows more than 200 sq miles of water
* Sort by zipcode (will match 21 rows)
* Create a GIST index on column `location`
* Use the Query Editor to run:
```sql
SELECT usps  
FROM census_zipcode_geography 
WHERE awater_sqmi > (SELECT avg(aland_sqmi) 
                     FROM census_zipcode_geography)
AND aland_sqmi > (SELECT avg(awater_sqmi)
                  FROM census_zipcode_geography)
ORDER BY location <-> point '(45.1,-69.3)'
LIMIT 20;
```
* Present the graphical explanation of the query plan
* Look at the statistics for the query plan
* Delete the index created

## psql
![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

As with the GUI tool, a walk through of using `psql` will make its usage much more clear than simply capturing screenshots or text portions of the interface.

* Login with 

```bash
$ psql -h localhost --username=datascience -W -p 5432 -d postgres
```
* Show SQL help (`\h`) and psql help (`\?`)
* List databases (`\l`)
* List users (`\du`)
* Create a new user `ine_test` with:
```sql
-- previously ran, e.g. `CREATE GROUP ine_users;`
postgres=# CREATE USER ine_test IN GROUP ine_users,practice CREATEDB PASSWORD 'ine-test';
```
* DROP the test user (synonym `DROP ROLE`)
* Connect to the database `ine` (`\c ine`)
* List the relations/tables (`\d`)
* DESCRIBE a table
* Perform the nested query in the GUI example
* Use EXPLAIN to understand the query plan
* Create an index on location:
```sql
CREATE INDEX index_location ON census_zipcode_geography USING GIST (location);
```
* EXPLAIN the query plan again
* List the indices (`\di`)
* Quit (`\q`)

Using `psql` is perhaps most powerful when you can use it as a scripting engine to pass along SQL from a file or from STDIN.  The `-f` switch can be used to take SQL statements from a file.  Piping from the command line also works.  Some examples below.  Note that to make this work without a password, you need to have credentials configured.  One way this is commonly done, if per system user, is to put credential in the file `~./pgpass`.

In [1]:
!cat ~/.pgpass


localhost:5432:ine:ine_student:ine-password


We might run a query from the shell (perhaps piping or redirecting the response).

In [2]:
!echo "SELECT * FROM users;" | psql -h localhost -U ine_student ine

 user_id | username |   password   | age |         created_on         
---------+----------+--------------+-----+----------------------------
       1 | Alice    | bad_pw_1     |  37 | 2020-11-29 00:33:42.202398
       2 | Bob      | bad_pw_2     |     | 2020-11-29 00:33:42.203537
       3 | Carlos   | bad_pw_3     |  62 | 2020-11-29 00:33:42.20391
      11 | Sybil    | M7c&sd31&0hA |  44 | 2020-11-29 00:44:39.883221
      12 | Trudy    | y9bD6SA2O%$t |  22 | 2020-11-29 00:44:39.883221
      13 | Vanna    | 9$Ts9HK*3!tR |  55 | 2020-11-29 00:44:39.883221
(6 rows)



Perhaps we might specialize it further.  Probably receivers of a pipe do not want the headers and status summary.

In [3]:
%%bash
export sql='SELECT row_to_json(users) FROM users LIMIT 2;'
echo $sql | psql -t -h localhost -U ine_student ine | jq -C

{
  "user_id": 1,
  "username": "Alice",
  "password": "bad_pw_1",
  "age": 37,
  "created_on": "2020-11-29T00:33:42.202398"
}
{
  "user_id": 2,
  "username": "Bob",
  "password": "bad_pw_2",
  "age": null,
  "created_on": "2020-11-29T00:33:42.203537"
}


In [4]:
%%bash 
sql='SELECT user_id, username, created_on FROM users;'
echo "$sql" | psql --csv -h localhost -U ine_student ine

user_id,username,created_on
1,Alice,2020-11-29 00:33:42.202398
2,Bob,2020-11-29 00:33:42.203537
3,Carlos,2020-11-29 00:33:42.20391
11,Sybil,2020-11-29 00:44:39.883221
12,Trudy,2020-11-29 00:44:39.883221
13,Vanna,2020-11-29 00:44:39.883221


## Summary
![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

As we go through more lessons of this course, you will become more familiar with PostgreSQL special capabilities and syntax.  This lesson should make you familiar with the general tools available to do actions like add, remove, or alter users, groups, schemas, databases, tables, and so on.  
All of those actions are very well accomplished either in the psql command shell, or if you prefer, in a GUI tool like pgAdmin.